In [4]:
from skmap.modeler import RFRegressor, RFRegressorTrees
from skmap.catalog import DataCatalog
from skmap.tiled_data import TiledData, TiledDataLoader, TiledDataExporter
from skmap.misc import TimeTracker, ttprint
import skmap_bindings as sb
import sys, os, warnings
import numpy as np
warnings.filterwarnings("ignore", module="sklearn")
import time
import gc

version = '20250212'

YEARS = range(2000, 2013, 2)
# YEARS = range(2012, 2023, 2)
DEPTHS = [0, 20, 50, 100, 200]
QUANTILES = [0.16, 0.84]

BASE_PATH = '/mnt/slurm/jobs/ai4sh_pred'
CATALOG_PATH = f'{BASE_PATH}/eu_soil_prop_v{version}.csv'
MODEL_PATH = f'{BASE_PATH}'
TILES_IDS = f'{BASE_PATH}/tiles_eu.in'
MASK_TEMPLATE_PATH = 'http://192.168.49.30:8333/ai4sh/masks/combined/{tile_id}.tif'
GDAL_OPTS = {'GDAL_HTTP_VERSION': '1.0', 'CPL_VSIL_CURL_ALLOWED_EXTENSIONS': '.tif'}
GAIA_ADDRS = [f'http://192.168.49.{gaia_ip}:8333' for gaia_ip in range(30, 47)]
THREADS = 96
DEPTH_VAR = 'hzn_dep'
RESAMPLING_STRATEGY = "GRA_CubicSpline"

S3_PARAMS = {
    's3_addresses':GAIA_ADDRS,
    's3_access_key':'iwum9G1fEQ920lYV4ol9',
    's3_secret_key':'GMBME3Wsm8S7mBXw3U4CNWurkzWMqGZ0n2rXHggS0',
    's3_prefix':f'tmp-ai4sh-layers/eu_props_v{version}',
}
# S3_PARAMS = None

MODE, MODEL_TYPE, MSF = ('depths_years_quantiles_textures', RFRegressorTrees, '.joblib')
# MODE, MODEL_TYPE, MSF = ('depths_years', RFRegressor, '.so')

# SPATIAL_AGGREGATION = 8
SPATIAL_AGGREGATION = None

spatial_res = f'{30*SPATIAL_AGGREGATION}m' if SPATIAL_AGGREGATION else '30m'
out_files_suffix = f'g_epsg.4326_v{version}'

start_tile=int(sys.argv[1])
end_tile=int(sys.argv[2])
server_name=sys.argv[3]

# start_tile = 100
# end_tile = 101
# server_name='ripley'


with open(TILES_IDS, 'r') as file:
    tile_ids = [line.strip() for line in file]
tile_ids = tile_ids[start_tile:end_tile]
base_dir = f'/mnt/{server_name}/ai4sh_pred_v2'


texture1_params = {
    'model':MODEL_TYPE(model_path=f'{MODEL_PATH}/model_rf.texture1_production_v{version}{MSF}',),
    'expm1':False, 'scale':1, 'nodata':255, 'dtype':'byte', 'prefix':'sand.tot_iso.11277.2020.wpct'
}
texture2_params = {
    'model':MODEL_TYPE(model_path=f'{MODEL_PATH}/model_rf.texture2_production_v{version}{MSF}',),
    'expm1':False, 'scale':1, 'nodata':255, 'dtype':'byte', 'prefix':'silt.tot_iso.11277.2020.wpct'
}
# Dummy model
texture3_params = {
    'model':MODEL_TYPE(model_path=f'{MODEL_PATH}/model_rf.texture2_production_v{version}{MSF}',),
    'expm1':False, 'scale':1, 'nodata':255, 'dtype':'byte', 'prefix':'clay.tot_iso.11277.2020.wpct'
}

assert(texture1_params['model'].n_trees == texture2_params['model'].n_trees)

models_params = [
    texture1_params,
    texture2_params,
    texture3_params
]

textures_features = {f for params in models_params for f in params['model'].model_covs}
catalog = DataCatalog.create_catalog(catalog_def=CATALOG_PATH, years=YEARS, base_path=GAIA_ADDRS, verbose=False)
YEARS_srt = [str(y) for y in YEARS]
catalog.query(textures_features, YEARS_srt)
textures_data = TiledDataLoader(catalog, MASK_TEMPLATE_PATH, SPATIAL_AGGREGATION, RESAMPLING_STRATEGY, verbose=False)

export_data = TiledDataExporter(spatial_res=spatial_res, s3_params=S3_PARAMS,
                                mode=MODE, years=YEARS, depths=DEPTHS, quantiles=QUANTILES)



Feature hzn_dep is missing in the original catalog, adding is in the otf (on the fly) common group
Added `g1` successfully.
Added `g2` successfully.
Added `g3` successfully.
Added `g4` successfully.
Added `g5` successfully.
Added `g6` successfully.
Added `g7` successfully.
Added `g8` successfully.
Added `g9` successfully.
Added `g10` successfully.
Added `g11` successfully.
Added `g12` successfully.
Added `g13` successfully.
Added `g14` successfully.
Added `g15` successfully.
Added `g16` successfully.
Added `g17` successfully.


In [5]:
for tile_id in tile_ids:
    print("--------------------------------------------------------------")
    
    export_data.tile_id = tile_id
    if export_data.check_all_exported([texture3_params['prefix'],
                                       texture1_params['prefix'],
                                       texture2_params['prefix']], 
                                       out_files_suffix):
        ttprint(f"All textures for tile {tile_id} already computed, skipping")
        continue

    with TimeTracker(f" o Reading data for tile {tile_id}", False):
        textures_data.load_tile_data(tile_id)
        if textures_data.n_pixels_valid == 0:
            ttprint("No pixels to predict in this tile, skipping")
            continue
        textures_data.convert_nan_to_median()
        textures_data.convert_nan_to_value(0.0)
        
    with TimeTracker(f" o Processing tile {tile_id}", False):
        texture1_model:MODEL_TYPE = texture1_params['model']
        texture2_model:MODEL_TYPE = texture2_params['model']
        with TimeTracker(f"     - Getting predictions", False):
            pred_depths_texture1 = []
            pred_depths_texture2 = []
            for depth in DEPTHS:
                with TimeTracker(f"       - Depth {depth}", False):
                    textures_data.fill_otf_constant(DEPTH_VAR, depth)
                    pred_depths_texture1 += [texture1_model.predict(textures_data)]
                    pred_depths_texture2 += [texture2_model.predict(textures_data)]
    with TimeTracker(f"   # Deriving statistics", False):
        if MODE == 'depths_years_quantiles_textures':
            export_data.derive_block_quantiles_and_mean_textures(pred_depths_texture1, pred_depths_texture2)
        elif MODE == 'depths_years':
            assert True, "Not available option" 
        del pred_depths_texture1
        del pred_depths_texture2
        gc.collect()
    with TimeTracker(f"   # Exporting files", False):
        export_data.export_files([texture3_params['prefix'],
                                  texture1_params['prefix'],
                                  texture2_params['prefix']],
                                  out_files_suffix,
                                  texture1_params['nodata'], textures_data.mask_path,
                                  texture1_params['dtype'], textures_data.get_pixels_valid_idx(1),
                                  write_folder=base_dir, scaling=texture1_params['scale'])
        del export_data.array
        gc.collect()
    textures_data.__exit__(None,None,None)
    print("--------------------------------------------------------------")



--------------------------------------------------------------
[01:20:37] All textures for tile 10R_24C already computed, skipping


In [1]:
import skmap_bindings as sb
from model_fit import textures_fw_transform, textures_bw_transform
from skmap.misc import sb_arr
import numpy as np
# t1 = np.ones((2,2), dtype=np.float32)
# t2 = np.ones((2,2), dtype=np.float32)

clay = sb_arr(2,2)
sand = sb_arr(2,2)
silt = sb_arr(2,2)

clay[0,0] = 100
sand[0,0] = 0
silt[0,0] = 0

clay[1,0] = 0
sand[1,0] = 100
silt[1,0] = 0

clay[0,1] = 5
sand[0,1] = 6
silt[0,1] = 89

clay[1,1] = 85
sand[1,1] = 6
silt[1,1] = 89

k = 1
a = 100

t1, t2 = textures_fw_transform(sand, silt, clay, k=k, a=a)
    
sand_trees, silt_trees, clay_trees = textures_bw_transform(t1, t2, k=k, a=a)

clay_trees_ = sb_arr(2,2)
sand_trees_ = sb_arr(2,2)
silt_trees_ = sb_arr(2,2)
        
sb.texturesBwTransform(t1, 1, t2, k, a, sand_trees_, silt_trees_, clay_trees_)

print(sand_trees_)
print(silt_trees_)
print(clay_trees_)
print("----------")

print(sand_trees)
print(silt_trees)
print(clay_trees)

[[  0.          5.9999943]
 [100.          0.       ]]
[[ 0.      89.00001]
 [ 0.      51.49254]]
[[100.          4.9999995]
 [  0.         48.507465 ]]
----------
[[  0.          5.9999943]
 [100.          0.       ]]
[[ 0.      89.00001]
 [ 0.      51.49254]]
[[100.          4.9999995]
 [  0.         48.507465 ]]
